In [1]:
# Imports
import torch
from torch.distributions import Normal,Laplace,StudentT
from dask_jobqueue import SLURMCluster
from distributed import Client
import BD_Cocycles,BD_SCM,BD_RKHS

In [2]:
# Args set up
N = 5000
ntrial = 10
batch_size = 64
scheduler = True
flip_prob = 0.05

hypers = ["weight_decay","batch_size","scheduler"]
hyper_vals = [[0,batch_size,scheduler],
              [1e-4,batch_size,scheduler],
              [1e-3,batch_size,scheduler],
              [1e-2,batch_size,scheduler],
              [1e-1,batch_size,scheduler]]

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10095",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
    #job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=100)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
    f1 = client.submit(BD_RKHS.run_experiment,seed,flip_prob,N)
    futures += [f1]
    for h in hyper_vals:
        f2 = client.submit(BD_Cocycles.run_experiment,seed,hypers,h,flip_prob,N)
        f3 = client.submit(BD_SCM.run_experiment,seed,Normal(0,1),False,hypers,h,flip_prob,N)
        f4 = client.submit(BD_SCM.run_experiment,seed,Laplace(0,1),False,hypers,h,flip_prob,N)
        f5 = client.submit(BD_SCM.run_experiment,seed,StudentT(10,0,1),True,hypers,h,flip_prob,N)
        futures += [f2,f3,f4,f5]

In [1]:
futures

NameError: name 'futures' is not defined

In [ ]:
# Getting results
results = client.gather(futures)

In [ ]:
# Closing client
client.close()
cluster.close()

In [ ]:
torch.save(f = "BD_results_new_N={4}_trials={0}_batchsize={1}_scheduler={2}_flip_prob={3}.pt".format(ntrial,batch_size,scheduler,flip_prob,N), obj = results)